In [77]:
import warnings
warnings.filterwarnings('ignore')
from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests

from bs4 import BeautifulSoup
import json
import re
import numpy as np

In [15]:
steamspy_url = 'https://steamspy.com/api.php?request=all'
payload = {'action': 'parse', 'format': 'json'}
r = requests.post(steamspy_url)

In [17]:
client = MongoClient('localhost', 27017)
db = client.steamspy
test = db.test

In [18]:
for item in r.json().values():
    test.insert(item)

In [19]:
data = pd.DataFrame(list(test.find()))

In [48]:
for idx in range(data.shape[0]):
    owners = data.iloc[idx]['owners'].split('..')
    for i in range(len(owners)):
        owners[i] = owners[i].replace(',','')
        owners[i] = owners[i].replace(' ','')
        owners[i] = int(owners[i])
    data.at[idx, 'llOwners'] = owners[0]
    data.at[idx, 'ulOwners'] = owners[1]

In [54]:
data = data.drop(columns = ['score_rank', 'owners', '_id'])

In [27]:
#Params: genre(string) represents the genre you want to get data of
#        collection(database collection) represents database to store the genre data in
#This function gets the data of a certain genre from the steamspy website and puts it in the given collection
def storeGenre(genre, collection):
    steamspy_url = 'https://steamspy.com/api.php?request=genre&genre={}'.format(genre)
    r = requests.post(steamspy_url)
    for item in r.json().values():
        collection.insert(item)

In [28]:
action = db.action

In [29]:
storeGenre('Action', action)

In [30]:
earlyaccess = db.earlyaccess
storeGenre('Early+Access', earlyaccess)

In [126]:
dataAction = pd.DataFrame(list(action.find()))

In [129]:
data['Genres'] = np.empty((len(df), 0)).tolist()

In [123]:
#Params: df(dataframe) represents the data to append to
#        genreData(dataframe) represents the genre dataframe that you are adding
#        genre(string) the genre you are appending
#This function appends to the Genre column of df if the game is in the certain genre
def appendGenre(df, genreData, genre):
    genreData = genreData.drop(columns = ['score_rank', 'owners', '_id'])
    df = pd.merge(df, genreData, on=['appid','average_2weeks','average_forever','developer',
                                   'discount', 'initialprice','median_2weeks','median_forever',
                                   'name', 'negative', 'positive', 'price', 'publisher',
                                   'userscore'], how='left', indicator='{}'.format(genre))
    for idx in range(df.shape[0]):
        if df.iloc[idx]['{}'.format(genre)] == 'both':
            df.at[idx, 'Genres'].append('{}'.format(genre))
    df = df.drop(columns = ['{}'.format(genre)])

In [130]:
appendGenre(data, dataAction, 'Action')